# Analysis notebook for CAB (with a focus on stimulus design/piloting)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
import pymongo as pm
import os, sys

In [ ]:
# display all columns
pd.set_option('display.max_columns', None)

## Create folders to save out results

In [ ]:
## create relevant project subdirs
proj_dir = os.path.abspath('..')
analysis_dir =  os.path.join(proj_dir,'analysis')
results_dir = os.path.join(proj_dir,'results')
csv_dir = os.path.join(results_dir,'csv')
plots_dir = os.path.join(results_dir,'plots')

def makedir(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return

In [ ]:
print("The project directory is: {}".format(proj_dir))

In [ ]:
makedir(results_dir)
makedir(csv_dir)
makedir(plots_dir)

In [ ]:
# we need to get to the repo level to import cabutils
sys.path.append(proj_dir)
sys.path.append(os.path.join(proj_dir,'stimuli'))
import cabutils
from stimuli.experiment_config import *

## Experiment parameters

In [ ]:
PROJECT = "Physion_V1_5" 
DATASET = "Dominoes"
TASK = "OCP"
ITERATION = "pilot_1"
EXPERIMENT = DATASET + "_" + TASK

## Load data from database

In [ ]:
!ssh -fNL 27017:127.0.0.1:27017 fbinder@cogtoolslab.org

In [ ]:
# this will create the database connection
from analysis.generate_dataframes import * 

In [ ]:
df, df_familiarization = pull_dataframes_from_mongo(PROJECT, DATASET, TASK, ITERATION, anonymizeIDs=False)

# Analysis

## Overall correctness

In [ ]:
# get percentage of correct trials
df['correct'].value_counts() / len(df) * 100

### Overall correctness by stimulus

In [ ]:
df.groupby('stim_ID')['correct'].mean()

In [ ]:
# histogram of avg correctness per stim
df.groupby('stim_ID')['correct'].mean().hist()
plt.title('Histogram of average correctness per stimulus')
plt.xlabel('n')
plt.ylabel('Average correctness')
plt.xlim(0,1)
plt.show()

### Overall correctness by participant

In [ ]:
# histogram of avg correctness per gameID
df.groupby('gameID')['correct'].mean().hist()
plt.title('Histogram of average correctness per participant')
plt.xlabel('n')
plt.ylabel('Average correctness')
plt.xlim(0,1)
plt.show()

## Reliability

In [ ]:
# import correlation metrics
from scipy.stats import pearsonr, spearmanr

In [ ]:
ITERATIONS = 10 # how many iterations to do **per n**
METRIC = lambda x,y: pearsonr(x,y)[0] # what metric to use for correlation

### Split half correlation over distinct halfs
This can only go up to n/2, where n is the number of trials.  

In [ ]:
correlations = {}
for n in tqdm(range(1,df['gameID'].nunique()//2)):
    correlations[n] = []
    for i in range(ITERATIONS):
        # sample n participants into two exclusive groups
        sample = df['gameID'].sample(n=n*2,replace=False)
        group1 = sample.iloc[:n]
        group2 = sample.iloc[n:]
        # get the correlation over average correctness per stimulus
        vec1 = df[df['gameID'].isin(group1)].groupby('stim_ID')['correct'].mean()
        vec2 = df[df['gameID'].isin(group2)].groupby('stim_ID')['correct'].mean()
        # possibly the groups contain different stimuli—drop the ones that don't overlap
        singular_stims = set(vec1.index).symmetric_difference(set(vec2.index))
        for sing in singular_stims:
            try: 
                vec1 = vec1.drop(sing)
            except: pass
            try:
                vec2 = vec2.drop(sing)
            except: pass
        assert np.all(list(vec1.index) == list(vec2.index))
        correlations[n].append(METRIC(vec1.values,vec2.values))

In [ ]:
# plot
plt.figure(figsize=(10,5))
plt.plot(list(correlations.keys()),[np.mean(correlations[n]) for n in correlations.keys()],label='mean')
plt.fill_between(list(correlations.keys()),[np.percentile(correlations[n],2.5) for n in correlations.keys()],
                    [np.percentile(correlations[n],97.5) for n in correlations.keys()],alpha=0.5,label='95% CI')
plt.title('Split half correlation between exclusive groups of participants')
plt.xlabel('Correlation coefficient')
plt.ylim(0,1)
plt.ylabel('n Participants')

### Bootstrapped split half correlation
Bootstrapping over participants. 

In [ ]:
correlations = {}
for n in tqdm(range(1,df['gameID'].nunique())):
    correlations[n] = []
    for i in range(ITERATIONS):
        # sample n participants into two exclusive groups
        group1 = df['gameID'].sample(n=n,replace=False)
        group2 = df['gameID'].sample(n=n,replace=False)
        # get the correlation over average correctness per stimulus
        vec1 = df[df['gameID'].isin(group1)].groupby('stim_ID')['correct'].mean()
        vec2 = df[df['gameID'].isin(group2)].groupby('stim_ID')['correct'].mean()
        # possibly the groups contain different stimuli—drop the ones that don't overlap
        singular_stims = set(vec1.index).symmetric_difference(set(vec2.index))
        for sing in singular_stims:
            try: 
                vec1 = vec1.drop(sing)
            except: pass
            try:
                vec2 = vec2.drop(sing)
            except: pass
        assert np.all(list(vec1.index) == list(vec2.index))
        correlations[n].append(METRIC(vec1.values,vec2.values))

In [ ]:
# plot
plt.figure(figsize=(10,5))
plt.plot(list(correlations.keys()),[np.mean(correlations[n]) for n in correlations.keys()],label='mean')
plt.fill_between(list(correlations.keys()),[np.percentile(correlations[n],2.5) for n in correlations.keys()],
                    [np.percentile(correlations[n],97.5) for n in correlations.keys()],alpha=0.5,label='95% CI')
plt.title('Split half correlation between groups of participants with overlap')
plt.xlabel('Correlation coefficient')
plt.ylim(0,1)
plt.ylabel('n Participants')

## Stimuli

Let's have an overview over all the stimuli

In [ ]:
df.groupby('stim_ID').agg({'correct':['mean','std'], 'target_hit_zone_label':'first'}).style.background_gradient(cmap='Oranges', vmin=0, vmax=1)

### The most extreme stimuli

In [ ]:
DISPLAY_N = 5

In [ ]:
from IPython.display import HTML

def display_rows(_df):
    """Expects a dataframe with the colums 'stimulus_name', 'response', 'stim_url'. 
    Needs to be wrapped in HTML() to display in a notebook"""
    html = ""
    for i,row in _df.iterrows():
        div = """
<div>
<b>Stim name</b>: {}<br>
<b>Predicted positive</b>: {}<br>
<b>True Outcome</b>:       {}<br>
<video width="40%" controls>
<source src="{}">
</video></div>""".format(row['stim_ID'],row['responseBool'],row['target_hit_zone_label'],row['mp4s_url'])
        html+=div
    return html

In [ ]:
# get the DISPLAY_N rows with the highest correctness
most_correct = df.groupby('stim_ID').agg({'correct':'mean', 'responseBool':'mean', 'mp4s_url':'first', 'target_hit_zone_label':'first'}).sort_values('correct',ascending=False).head(DISPLAY_N).reset_index()
least_correct = df.groupby('stim_ID').agg({'correct':'mean', 'responseBool':'mean', 'mp4s_url':'first', 'target_hit_zone_label':'first'}).sort_values('correct',ascending=True).head(DISPLAY_N).reset_index()

The **most correct** stimuli

In [ ]:
HTML(display_rows(most_correct))

The **least correct** stimuli

In [ ]:
HTML(display_rows(least_correct))